In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import random
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [ ]:
df_gt   = pd.read_excel("drive/My Drive/Projects/LSE-Symptomatology/Dataset/Baseline Comparisons/PCOS Symptoms (Groud Truth).xlsx")[['postID', 'PCOS Symptom (Grd Truth)']]
df_chatgpt   = pd.read_excel("drive/My Drive/Projects/LSE-Symptomatology/Dataset/Baseline Comparisons/PCOS Symptoms (ChatGPT-4o).xlsx")[['postID', 'LLM2(GPT 4o)']]
df_lse    = pd.read_excel("drive/My Drive/Projects/LSE-Symptomatology/Dataset/Baseline Comparisons/PCOS Symptoms (LSE).xlsx")[['postID', 'LSE-guided LLM(GPT-4o)']]
df_llama  = pd.read_excel("drive/My Drive/Projects/LSE-Symptomatology/Dataset/Baseline Comparisons/PCOS Symptoms (Llama 3.1).xlsx")[['postID', 'LLM1(Llama-3.1-8B-Instruct)']]
df_google = pd.read_excel("drive/My Drive/Projects/LSE-Symptomatology/Dataset/Baseline Comparisons/PCOS Symptoms (Google Health).xlsx")[['postID', 'GHNLP']]
df_aws    = pd.read_excel("drive/My Drive/Projects/LSE-Symptomatology/Dataset/Baseline Comparisons/PCOS Symptoms (Amazon Comprehend Medical).xlsx")[['postID', 'ACM']]


In [ ]:
df = pd.concat([
    df_gt.set_index('postID'),
    df_chatgpt.set_index('postID'),
    df_lse.set_index('postID'),
    df_llama.set_index('postID'),
    df_google.set_index('postID'),
    df_aws.set_index('postID'),
], axis=1).reset_index()

In [ ]:
df

,postID,PCOS Symptom (Grd Truth),LLM2(GPT 4o),LSE-guided LLM(GPT-4o),LLM1(Llama-3.1-8B-Instruct),GHNLP,ACM
0,xapb9j,none,none,none,none,none,none
1,va1s9b,weight gain,none,weight gain,"acne, excess hair growth, weight gain, irregul...","losing weight, unhealthy","gaining weight, losing weight, stabilized, unh..."
2,vyv2ut,none,none,none,none,none,none
3,wy5f9k,cysts on overies,cysts on ovaries,cysts on ovaries,cysts on ovaries.,"cysts, follicles","cysts, follicles, regular periods"
4,yvjy63,hair loss,hair loss,hair loss,hair loss.,"balding, hair loss","affecting my mental health, balding, hair loss"
...,...,...,...,...,...,...,...
995,y87em5,"hair loss, weight gain, acne, cramps, insulin ...","hair loss on scalp, weight gain, acne along ja...","hair loss, weight gain, acne, cramps, insulin ...","hair loss on scalp, weight gain, acne along ja...","acne, anxiety, cramps, depression, facial swe...","acne, anxiety, cramps, depression, elevated C..."
996,v96ws6,"irregular periods, weight gain, high blood pre...","irregular periods, weight gain, circulation co...","irregular periods, insulin resistance, weight ...","Weight gain, nausea, lack of appetite, consist...","anxious, concerns, irregular periods, lack of...","affect my fertility, anxious, high BP., insul..."
997,yj6zyi,"hirsutism, acne, irregular periods, painful cr...","excess hair growth, acne, irregular periods, v...","irregular periods, hirsutism, acne, painful pe...","excess hair growth, acne, irregular periods, v...","bloated, constipated, diarrhoea, digestive is...","IBS, acne, bloating, bowel movements, constip..."
998,y6k0n6,"hirsutism, acne,weight gain, low energy, hot f...","weight gain, hirsutism, acne, irregular menstr...","weight gain, hirsutism, acne, irregular period...","hirsutism, acne, hot flashes, low energy, weig...","Hormone irregularity, Hot flashes, acne, anxie...","Hormone irregularity, Hot flashes, Low energy,..."


In [ ]:
#display dataframe
df['LLM2(GPT 4o)'][424]

nan

In [ ]:
# Define a similarity threshold
similarity_threshold = 0.6

In [ ]:
# Load SapBERT model
with open('BioBERT.txt', 'r') as f:
    model_name = f.read().strip()
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# Function to encode symptoms
def get_embeddings(symptom_list, model, tokenizer):
    embeddings = []
    for symptom in symptom_list:
        # Tokenize and encode
        inputs = tokenizer(symptom, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the [CLS] token representation as the embedding
        embeddings.append(outputs.last_hidden_state[:, 0, :])  # Shape: (1, hidden_size)
    return torch.vstack(embeddings)

In [ ]:
def process_symptoms(symptoms):
    # processes symptoms
    symptoms = str(symptoms) if not pd.isna(symptoms) else "none"
    return [symptom.strip().lower().replace('.', '') for symptom in symptoms.split(',') if symptom.strip()]

In [ ]:
def process_duplicates(method_init_symptoms):
    method_init_embeddings = get_embeddings(method_init_symptoms, model, tokenizer).numpy() # Encode all initial symptoms
    method_a_symptoms = set()
    method_a_embeddings = []  # Initialize an empty list to store embeddings of non-duplicate symptoms

    for index, symptom in enumerate(method_init_symptoms):
        if not method_a_symptoms:  # If set is empty, add the first symptom
            method_a_symptoms.add(symptom)
            method_a_embeddings.append(method_init_embeddings[index])
        else:
            is_duplicate = False
            # Compare the symptom's embedding with all embeddings in the non-duplicate set
            for method_a_embedding in method_a_embeddings:
                similarity_scores = cosine_similarity([method_init_embeddings[index]], [method_a_embedding])
                if np.max(similarity_scores) >= 0.8:
                    is_duplicate = True
                    break
            if not is_duplicate:
                method_a_symptoms.add(symptom)
                method_a_embeddings.append(method_init_embeddings[index])

    return list(method_a_symptoms)

In [ ]:
# Function to calculate F1-score
def calculate_f1_scores(ground_truth, predicted):
    correct_matches = 0
    ground_truth_embeddings = get_embeddings(ground_truth, model, tokenizer).numpy()
    predicted_embeddings = get_embeddings(predicted, model, tokenizer).numpy()
    used_ground_truth = set()


    for predicted_embedding in predicted_embeddings:
        similarity_scores = cosine_similarity([predicted_embedding], ground_truth_embeddings)
        max_similarity_score_index = np.argmax(similarity_scores)
        max_similarity_score = similarity_scores[0][max_similarity_score_index]

        if max_similarity_score >= similarity_threshold and max_similarity_score_index not in used_ground_truth:
            correct_matches += 1
            used_ground_truth.add(max_similarity_score_index)

    missed_symptoms = len(ground_truth) - correct_matches
    wrong_matches = len(predicted) - correct_matches

    precision = correct_matches / len(predicted) if len(predicted) > 0 else 0
    recall = correct_matches / len(ground_truth) if len(ground_truth) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return f1_score

In [ ]:
def identify_unique(method_a_symptoms, method_b_symptoms):
    # Encode the symptoms using the BERT model

    method_a_embeddings =   get_embeddings(method_a_symptoms, model, tokenizer).numpy()
    method_b_embeddings =   get_embeddings(method_b_symptoms, model, tokenizer).numpy()

    # Initialize sets to store unique symptoms
    unique_a = set(method_a_symptoms)
    unique_b = set(method_b_symptoms)

    # Compare each symptom in method A with all symptoms in method B using cosine similarity
    for idx_a, method_a_embedding in enumerate(method_a_embeddings):
        for idx_b, method_b_embedding in enumerate(method_b_embeddings):
            similarity_score = cosine_similarity([method_a_embedding], [method_b_embedding])[0][0]

            # If similarity exceeds the threshold, remove symptoms from unique sets
            if similarity_score >= similarity_threshold:
                if method_a_symptoms[idx_a] in unique_a:
                    unique_a.remove(method_a_symptoms[idx_a])
                if method_b_symptoms[idx_b] in unique_b:
                    unique_b.remove(method_b_symptoms[idx_b])

    return unique_a, unique_b

In [ ]:
# Common outputs (Common symptoms of method A and B)
def identify_common(method_a_symptoms, method_b_symptoms):

    # Encode the symptoms using the BERT model
    method_a_embeddings =   get_embeddings(method_a_symptoms, model, tokenizer).numpy()
    method_b_embeddings =   get_embeddings(method_b_symptoms, model, tokenizer).numpy()

    common_symptoms = set()

    # Compare each symptom in method A with all symptoms in method B using cosine similarity
    for idx, method_a_embedding in enumerate(method_a_embeddings):
        similarity_scores = cosine_similarity([method_a_embedding], method_b_embeddings)

        # Check if the maximum similarity score exceeds the threshold and add to common set if true
        if np.max(similarity_scores) >= similarity_threshold:
            common_symptoms.add(method_a_symptoms[idx])  # Adding the symptom from method A considered as matching

    return common_symptoms


In [ ]:
# Initializtion
p_values = []
f1_scores_lse = []
f1_scores_ghnlp = []
f1_scores_acm = []
f1_scores_llm1 = []
f1_scores_llm2 = []

In [ ]:
start_index = 0
end_index = 1000

In [ ]:
# Process each row in the dataframe
#for index, row in df.iterrows():
for index, row in df.iloc[start_index:end_index].iterrows():

    ground_truth = process_symptoms(row['PCOS Symptom (Grd Truth)'])
    our_method_symptoms = process_symptoms(row['LSE-guided LLM(GPT-4o)'])
    #llm1_symptoms = process_symptoms(row['LLM1(Llama-3.1-8B-Instruct)'])
    llm2_symptoms = process_symptoms(row['LLM2(GPT 4o)'])


    #initial symptoms from dataframe
    #init_google_health_symptoms = process_symptoms(row['GHNLP'])
    #init_acm_symptoms = process_symptoms(row['ACM'])

    #process duplicates to identify final symptoms
    #google_health_symptoms = process_duplicates(init_google_health_symptoms)
    #acm_symptoms = process_duplicates(init_acm_symptoms)



    # Calculate F1-score for Our Method and Google Health
    f1_score_our_method = calculate_f1_scores(ground_truth, our_method_symptoms)
    #f1_score_google_health = calculate_f1_scores(ground_truth, google_health_symptoms)
    #f1_score_acm = calculate_f1_scores(ground_truth, acm_symptoms)
    #f1_score_llm1 = calculate_f1_scores(ground_truth, llm1_symptoms)
    f1_score_llm2 = calculate_f1_scores(ground_truth, llm2_symptoms)

    #append f1-scores
    f1_scores_lse.append(f1_score_our_method)
    #f1_scores_ghnlp.append(f1_score_google_health)
    #f1_scores_acm.append(f1_score_acm)
    # f1_scores_llm1.append(f1_score_llm1)
    f1_scores_llm2.append(f1_score_llm2)

    # Compute delta observed
    #delta_observed = f1_score_our_method - f1_score_google_health
    #delta_observed = f1_score_our_method - f1_score_acm
    #delta_observed = f1_score_our_method - f1_score_llm1
    delta_observed = f1_score_our_method - f1_score_llm2

    # Identify common and unique symptoms
    #common_symptoms = identify_common(our_method_symptoms, google_health_symptoms)
    #unique_symptoms_a, unique_symptoms_b = identify_unique(our_method_symptoms, google_health_symptoms)

    #common_symptoms = identify_common(our_method_symptoms, acm_symptoms)
    #unique_symptoms_a, unique_symptoms_b = identify_unique(our_method_symptoms, acm_symptoms)

    #common_symptoms = identify_common(our_method_symptoms, llm1_symptoms)
    #unique_symptoms_a, unique_symptoms_b = identify_unique(our_method_symptoms, llm1_symptoms)

    common_symptoms = identify_common(our_method_symptoms, llm2_symptoms)
    unique_symptoms_a, unique_symptoms_b = identify_unique(our_method_symptoms, llm2_symptoms)

    # Randomization test
    num_simulations = 100
    count = 0
    combined = list(unique_symptoms_a | unique_symptoms_b)

    #valid_simulations=0
    for _ in range(num_simulations):
        PA_prime = set()
        PB_prime = set()

        for symptom in combined:
            if random.choice([True, False]):  # 50% chance
                PA_prime.add(symptom)
            else:
                PB_prime.add(symptom)

        OA_prime = PA_prime | common_symptoms
        OB_prime = PB_prime | common_symptoms

        if(OA_prime== set() or OB_prime== set()):
            continue



        f1_prime_methodA = calculate_f1_scores(ground_truth, list(OA_prime))
        f1_prime_methodB = calculate_f1_scores(ground_truth, list(OB_prime))
        delta_prime = f1_prime_methodA - f1_prime_methodB

        if delta_prime > delta_observed:
            count += 1

    # Calculate p-value for this row
    p_value = count / num_simulations
    p_values.append(p_value)





In [ ]:
significant_cases_95 = 0
significant_cases_90 = 0

In [ ]:

for p_value in p_values:

  if p_value < 0.05:
        significant_cases_95 += 1
  if p_value < 0.1:
        significant_cases_90 += 1


In [ ]:
significant_cases_95, significant_cases_90 #[LSE vs ACM]

In [ ]:
significant_cases_95, significant_cases_90 #[LSE vs GHNLP]

In [ ]:
significant_cases_95, significant_cases_90 #[LSE vs Llama]

In [ ]:
significant_cases_95, significant_cases_90 #[LSE vs GPT-4o]

(145, 149)

In [ ]:
index

699

In [ ]:
avg_f1_score_lse = np.mean(f1_scores_lse)
#avg_f1_score_ghnlp = np.mean(f1_scores_ghnlp)
#avg_f1_score_acm = np.mean(f1_scores_acm)
#avg_f1_scores_llm1 = np.mean(f1_scores_llm1)
avg_f1_scores_llm2 = np.mean(f1_scores_llm2)

In [ ]:
avg_f1_score_lse

0.9066553870965635

In [ ]:
avg_f1_scores_llm2

0.661020864756159

In [ ]:
p_values

[0.0,
 0.497,
 0.0,
 0.252,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.488,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.066,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.306,
 0.0,
 0.763,
 0.0,
 0.131,
 0.628,
 0.25,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.03,
 0.0,
 0.0,
 0.355,
 0.498,
 0.243,
 0.506,
 0.76,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.515,
 0.062,
 0.0,
 0.509,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.253,
 0.0,
 0.452,
 0.0,
 0.259,
 1.0,
 0.0,
 0.876,
 0.0,
 0.0,
 0.499,
 0.0,
 0.0,
 0.0,
 0.368,
 0.0,
 0.0,
 0.0,
 0.523,
 0.0,
 0.0,
 0.0,
 0.119,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.722,
 0.123,
 0.0,
 0.0,
 0.754,
 0.0,
 1.0,
 0.0,
 0.0,
 0.489,
 0.0,
 0.0,
 0.0,
 0.255,
 0.0,
 0.607,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.522,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.506,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,

In [ ]:
# Calculate the average p-value
#avg_p_value = round(np.mean(p_values), 2)
avg_p_value= np.mean(p_values)
print("Average p-value:", avg_p_value)
if avg_p_value < 0.1:
    print("The difference is statistically significant on average.")
else:
    print("The difference is not statistically significant on average.")


Average p-value: 0.11339999999999999
The difference is not statistically significant on average.
